In [1]:
import os
import urllib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
today = datetime.now().strftime('%Y-%m-%d')
yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
month_year = datetime.now().strftime('%b-%Y')
z = r'C:\Users\student\Documents'

In [3]:
csv = (r'C:\Users\student\Documents\QAQC_reports\csv_vessel' + r'csv.csv')

In [4]:
def QAQC_datapull(a,b,c,d):
    '''
    Sends a data request to the CoAgMET server and processes the data into
    an easy to read dataframe
    
    Arguments
    - a = temporal frequency of the data 
        -must be entered as a string
        -options:
            -'five_minute'
            -'hourly'
            -'daily'
    - b = a five character ID for one or more of the 87 CoAgMET stations
        -must be entered as a string
        -when calling data from multiple stations separate stations IDs
        with a comma
        -Station IDs can be found at https://coagmet.colostate.edu/station_index.php
    - c = first day of the period from which you would like to request data
        -must be entered as a string in 'yyyy-mm-dd' format
    - d = last day of the period from which you would like to request data
        -must be entered as a string in 'yyyy-mm-dd' format
    '''
    # Determines which set of elements to request depending on the data type
    if a == 'hourly':
        e = 'tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir'
        r = pd.date_range(start=c,end=d,freq='H')
    elif a == 'five_minute':
        e = 'tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir'
        r = pd.date_range(start=c,end=d,freq='5min')
    elif a == 'daily':
        e = 'tave,tmax,tmin,vp,rhmax,rhmin,sr,wrun,pp,' + 
        'st5mx,st5mn,st15mx,st15mn,gust,gustdir'
        r = pd.date_range(start=c,end=d,freq='D')
        
    try:
        # Makes the data request and loads the data into a csv
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?' +
            'type=' + a +
            '&sids=' + b +
            '&sdate=' + c +
            '&edate=' + d +
            '&elems=' + e,
            filename=csv)
    except Exception as e:
        print('Something went wrong while pulling data from ' + 
              b + ': ' + e.args[0])
    
    
    # Loads the data from the csv into a pandas DataFrame
    data = pd.read_csv(csv)
    data = data.reset_index()

    # Converts the elements string into a list so they can be used 
    # as headers in the dataframe
    headers = e.split(',')
    # Adds to objects to the beginning of list for columns that are added by default
    headers.insert(0,'date')
    headers.insert(0,'station')
    # Assigns the objects in the list as column headers in the dataframe
    data.columns = headers
    
    # Instructs the computer to recognize values in the 'date' column are timestamps
    data['date'] = pd.to_datetime(data.date)
    
    # Creates a datetime index depending on the temporal frequency
    # of the data requested
    if a == 'hourly':
        r = pd.date_range(start=c,end=d,freq='H')
    elif a == 'five_minute':
        r = pd.date_range(start=c,end=d,freq='5min')
    elif a == 'daily':
        r = pd.date_range(start=c,end=d,freq='D')
    
    # Reindexes the CoAgMET data against the newly created datetime index to
    # make it easier to account for missing data
    data = data.set_index('date').reindex(r,copy=False).rename_axis('date')

    # Ensures that all values in the 'station' column are correct
    data['station'] = b
    
    return data

In [5]:
def create_report(x):
    # Stores a path in which te report will be stored
    y = x['path'] + r'\_' + month_year
    # Creates the pathway if it doesn't already exist
    if not os.path.exists(y): os.makedirs(y)
    # Creates a full name for the file that will be created    
    file = (y + r'\_' + yesterday + '_' + x['drainage'] + 
            '_' + x['region'] + '_QAQC_report.txt')
    return file

In [6]:
# A set of dictionaries for each region in the QAQC app
Collegiate_Valley = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\Collegiate_Valley'),
    'stations':'bnv01,sld01',
    'drainage':'Arkansas',
    'region':'Collegiate_Valley'
}
East_Valley_Bottom = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\East_Valley_Bottom'),
    'stations':'hly01,hly02,lam01,lam03,lam04,mcl01',
    'drainage':'Arkansas',
    'region':'East_Valley_Bottom'
}
Mesas = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\Mesas'),
    'stations':'cnn01,hne01,pnr01,wcf01',
    'drainage':'Arkansas',
    'region':'Mesas'
}
North_of_Valley = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\North_of_Valley'),
    'stations':'scm01',
    'drainage':'Arkansas',
    'region':'Mesas'
}
Plateau_South_of_Valley = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\Plateau_South_of_Valley'),
    'stations':'wls01',
    'drainage':'Arkansas',
    'region':'Plateau_South_of_Valley'
}
West_Valley_Bottom = {
    'path':(z + r'\QAQC_reports\Arkansas_Drainage\West_Valley_Bottom'),
    'stations':'avn01,fwl01,ljt01,rfd01',
    'drainage':'Arkansas',
    'region':'West_Valley_Bottom'
}
Lower_Valley = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Lower_Valley'),
    'stations':'cbl01,slt01',
    'drainage':'Colorado',
    'region':'Lower_Valley'
}
Eagle_River_Valley = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Eagle_River_Valley'),
    'stations':'egl01,gyp01',
    'drainage':'Colorado',
    'region':'Eagle_River_Valley'
}
Four_Corners = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Four_Corners'),
    'stations':'ctz01,dvc01,mnc01,twc01,yjk01,yuc01',
    'drainage':'Colorado',
    'region':'Four_Corners'
}
Grand_Valley = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Grand_Valley'),
    'stations':'cbn01,frt03,orm02',
    'drainage':'Colorado',
    'region':'Grand_Valley'
}
Headwaters = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Headwaters'),
    'stations':'gby01,krm01,wfd01',
    'drainage':'Colorado',
    'region':'Headwaters'
}
Lower_Gunnison = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Lower_Gunnison'),
    'stations':'dlt01,ekt01,hot02,mtr01,oth01',
    'drainage':'Colorado',
    'region':'Lower_Gunnison'
}
San_Juan = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\San_Juan'),
    'stations':'drg01,ign01,kln01,pgs01',
    'drainage':'Colorado',
    'region':'San Juan'
}
San_Miguel_Paradox = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\San_Miguel_Paradox'),
    'stations':'brk01,nwd01',
    'drainage':'Colorado',
    'region': r'San_Miguel_Paradox'
}
Upper_Gunnison = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Upper_Gunnison'),
    'stations':'gun01',
    'drainage':'Colorado',
    'region':'Upper_Gunnison'
}
Yampa_White = {
    'path':(z + r'\QAQC_reports\Colorado_Drainage\Yampa_White'),
    'stations':'clk01,hyd01,mkr01',
    'drainage':'Colorado',
    'region': 'Yampa_White'
}
North_Plains = {
    'path':(z + r'\QAQC_reports\Kansas_Drainage\North_Plains'),
    'stations':'akr02,hxt01,hyk02,ilf01,pai01,stg01,wry02,yum02',
    'drainage':'Kansas',
    'region':'North_Plains'
}
South_Plains = {
    'path':(z + r'\QAQC_reports\Kansas_Drainage\South_Plains'),
    'stations':'brl02,brl03,idl01,sbt01,stn01',
    'drainage':'Kansas',
    'region':'South_Plains'
}
Front_Range_Foothills = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\Front_Range_Foothills'),
    'stations':'ckp01',
    'drainage':'Platte',
    'region':'Front_Range_Foothills'
}
Lower_Plains = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\Lower_Plains'),
    'stations':'brg01,hxt01,ksy01,ksy02',
    'drainage':'Platte',
    'region':'Lower_Plains'
}
North_Front_Range = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\North_Front_Range'),
    'stations':'alt01,fcc01,fcl01,ftc01,ftc03,gly04,lcn01',
    'drainage':'Platte',
    'region':'North_Front_Range'
}
North_Park = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\North_Park'),
    'stations':'cow01,heb01,lar01',
    'drainage':'Platte',
    'region':'North_Park'
}
South_Front_Range = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\South_Front_Range'),
    'stations':'ccr01,cht01,eac01,ftl01,lsl01,pkh01',
    'drainage':'Platte',
    'region':'South_Front_Range'
}
South_Park = {
    'path':(z + r'\QAQC_reports\Platte_Drainage\South_Park'),
    'stations':'bnv01,jfn01,sld01',
    'drainage':'Platte',
    'region':'South_Park'
}
San_Luis_Valley = {
    'path':(z + r'\QAQC_reports\Rio_Grande_Drainage\San_Luis_Valley'),
    'stations':'bla01,ctr01,ctr02,ljr01,san01',
    'drainage':'Rio_Grande',
    'region':'San_Luis_Valley'
}


region_list = [Collegiate_Valley,East_Valley_Bottom,Mesas,North_of_Valley,
               Plateau_South_of_Valley,West_Valley_Bottom,Lower_Valley,
               Eagle_River_Valley,Four_Corners,Grand_Valley,Headwaters,
               Lower_Gunnison,San_Juan,San_Miguel_Paradox,Upper_Gunnison,
               Yampa_White,North_Plains,South_Plains,Front_Range_Foothills,
               Lower_Plains,North_Front_Range,North_Park,South_Front_Range,
               South_Park,San_Luis_Valley]

In [7]:
def QC_report_by_region(x):
    '''
    Runs through the previous days data each region of the QC and reports errors 
    into a text file
    '''
    # Calls the 'create_report' function to open a new text file 
    # and report on yesterdays data
    file = create_report(x)
    f = open(file,'w+')
    
    # Stores a title for each QA/QC report
    title = (yesterday + r' QA/QC Report for stations in the ' + x['region'] +
            ' region of the ' + x['drainage'] + ' drainage\n')
    
    # Writes the 'title' to the first line of the text file
    f.write(title)
    f.write('\n')
    
    # Creates an empty dataframe into which all of the five minute data
    # from each region will be entered
    fmdf = pd.DataFrame()
    # Turns the 'stations' item in each dictionary into a list separated by commas
    station_list = x['stations'].split(',')
    
    f.write('Five Minute Data\n')
    f.write('------------------\n')
    f.write('Missing Data:\n')
    f.write('----------------\n')
    
    # Uses the newly created 'station_list' to loop through all of the
    # stations in a given region and calls the 'QAQC_datapull' function 
    # to collect all of yesterdays five minute data into a DataFrame
    for x in station_list:
        try:
            fmdf = fmdf.append(QAQC_datapull('five_minute',x,yesterday,today))
        except ValueError:
            f.write(x + ' is missing all five minute data\n')
    
    # Right now the dataframe is using a datatime index, 
    # but the function has called all of yesterdays data from mulitple stations
    # leaving us with duplicate index values.  To remedy this issue, 
    # the DataFrames index will be reset to default so that the computer 
    # can identify individual rows.
    fmdf = fmdf.reset_index()
            
    # Loops through the entire dataframe to find and report missing values
    # to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if(pd.isnull(fmdf['tmean'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing temperature data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['rh'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing relative humidity data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['vp'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing vapor pressure data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['sr'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing solar radiation data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['ws'][n])):
                f.write(str(fmdf['station'][n]) + 
                       ' is missing wind speed data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['wind_vec'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing wind direction data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['wind_std'][n])):
                f.write(str(fmdf['station'][n]) + 
                       ' is missing wind direction standard deviation data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['pp'][n])):
                f.write(str(fmdf['station'][n]) + 
                       ' is missing precipitation data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['st5'][n])):
                f.write(str(fmdf['station'][n]) + 
                       ' is missing 5cm soil temperature data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['st15'][n])):
                f.write(str(fmdf['station'][n]) + 
                       ' is missing 15cm soil temperature data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['gust'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing gust speed data at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif(pd.isnull(fmdf['gustdir'][n])):
                f.write(str(fmdf['station'][n]) + 
                        ' is missing gust direction data at ' + 
                        str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for missing five minute data: ' +
              e.args[0])
    f.write('\n')
    f.write('\n')
        
    f.write('Temperature Data Report\n')
    f.write('-----------------------\n')
    
    # Loops through all temperature data to find a report errors to the text file
    try:                
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['tmean'][n] > 46.1:
                f.write(str(fmdf['station'][n]) + 
                        ' reported a temperature of ' + str(fmdf['tmean'][n]) + 
                        '°C on ' + str(fmdf['date'][n]) + 
                        '. If true, this a new state record for max temp.\n')
            elif fmdf['tmean'][n] < -51.7:
                f.write(str(fmdf['station'][n]) + 
                        ' reported a temperature of ' + str(fmdf['tmean'][n]) +
                       '°C on ' + str(fmdf['date'][n]) + 
                        '. If true, this a new state record for min temp.\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute temperature data: ' + 
              e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Relative Humidity Data Report\n')
    f.write('------------------------------\n')
    
    # Loops throught the relative humidity data to find and
    # report errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['rh'][n] > 1.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported a relative humidity of ' + str(fmdf['rh'][n]) +
                        ' at ' + str(fmdf['date'][n]) + '\n')
            elif fmdf['rh'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported negative relative humidity at ' +
                       str(fmdf['date'][n]) + '\n')
            elif fmdf['rh'][n] < 0.01:
                f.write(str(fmdf['date'][n]) + 
                        ' broke the world record for low relative humidity at ' + 
                       str(fmdf['station'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute relative humidity data: ' + 
              e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Vapor Pressure Data Report\n')
    f.write('--------------------------\n')
    #
    #
    #
    #
    
    f.write('\n')
    f.write('\n')
    
    f.write('Solar Radiation Data Report\n')
    f.write('---------------------------\n')
    
    # Loops through solar radiation data to find and report errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['sr'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported negative solar radiation at ' + 
                        str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute solar radiation data: ' + 
              e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Speed Data Report\n')
    f.write('----------------------\n')
    
    # Loops through wind speed data to find and report errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['ws'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported negative wind speed at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif fmdf['ws'][0] > 77.785:
                f.write(str(fmdf['station'][n]) + 
                        ' reported a record breaking wind speed at ' + 
                        str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute windspeed data: ' + 
              e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Direction Data Report\n')
    f.write('--------------------------\n')
    
    # Loops thorugh wind direction data to find and report errors to the text file
    try:
        n = 0 
        for index,rows in fmdf.iterrows():
            if fmdf['wind_vec'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported negative wind direction at ' +
                        str(fmdf['date'][n]) + '\n')
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute wind direction data: ' + 
              e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Precipitation Data Report\n')
    f.write('-------------------------\n')
    
    # Loops through precipitation data to find and report errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['pp'][n] > 51.562:
                f.write(str(fmdf['station'][n]) + 
                        ' reported recording breaking amount of precip at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif fmdf['pp'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + ' reported negative precip at ' + 
                        str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute precip data: ' +
              e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('5cm Soil Temperature Data Report\n')
    f.write('--------------------------------\n')
    
    # Loops through the 5cm soil temperature data to find 
    # and report any errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['st5'][n] > 50.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported impossibly high 5cm soil temperature at ' +
                       str(fmdf['date'][n]) + '\n')
            elif fmdf['st5'][n] < -15.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported impossibly low 5cm soil temperature at ' + 
                       str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for errors' + 
              ' in five minute 5cm soil temperature data: ' 
              + e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('15cm Soil Temperature Data Report\n')
    f.write('---------------------------------\n')
    
    # Loops through the 15cm soil temperature data to find
    # and report any errors the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['st15'][n] > 50.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported impossibly high 15cm soil temeprature at ' + 
                       str(fmdf['date'][n]) + '\n')
            elif fmdf['st15'][n] < -15.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported impossibly low 15cm soil temperature at ' + 
                       str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print(
            'Something went wrong while looking for' + 
            ' errors in five minute 15cm soil temperatures: ' + 
            e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Speed Data Report\n')
    f.write('----------------------\n')
    
    # Loops through gust speed data for find and report any errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['gust'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported negative gust speed at ' + 
                        str(fmdf['date'][n]) + '\n')
            elif fmdf['gust'][n] > 77.785:
                f.write(str(fmdf['station'][n]) + 
                        ' reported a record breaking gust speed at ' + 
                        str(fmdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in five minute gust speed data: ' + 
              e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Direction Data Report\n')
    f.write('--------------------------\n')
    
    # Loops through gust direction data to find and report 
    # any errors to the text file
    try:
        n = 0
        for index,rows in fmdf.iterrows():
            if fmdf['gustdir'][n] < 0.0:
                f.write(str(fmdf['station'][n]) + 
                        ' reported negative gust direction at ' + 
                        str(fmdf['date'][n]) + '\n' )
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for errors' + 
              ' in five minute gust direction data: ' + 
              e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    # Creates an empty dataframe into which all of the hourly data from each region 
    # will be entered
    hdf = pd.DataFrame()
    
    f.write('Hourly Data\n')
    f.write('------------\n')
    f.write('Missing data\n')
    f.write('------------\n')
    
    # Uses the newly created 'station_list' to loop through 
    # all of the stations in a given region and calls the 
    # 'QAQC_datapull' function to collect all of yesterdays hourly data 
    # into a DataFrame
    for x in station_list:
        try:
            hdf = hdf.append(QAQC_datapull('hourly',x,yesterday,today))
        except ValueError:
            f.write(x + ' is missing all hourly data\n')
    
    hdf = hdf.reset_index()

    try:   
        n = 0
        for index,rows in hdf.iterrows():
            if(pd.isnull(hdf['tmean'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing temperature data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['rh'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing relative humidity data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['vp'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing vapor pressure data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['sr'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing solar radiation data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['ws'][n])):
                f.write(str(hdf['station'][n]) + 
                       ' is missing wind speed data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['wind_vec'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing wind direction data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['wind_std'][n])):
                f.write(str(hdf['station'][n]) + 
                       ' is missing wind direction standard deviation data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['pp'][n])):
                f.write(str(hdf['station'][n]) + 
                       ' is missing precipitation data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['st5'][n])):
                f.write(str(hdf['station'][n]) + 
                       ' is missing 5cm soil temperature data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['st15'][n])):
                f.write(str(hdf['station'][n]) + 
                       ' is missing 15cm soil temperature data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['gust'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing gust speed data at ' + 
                        str(hdf['date'][n]) + '\n')
            elif(pd.isnull(hdf['gustdir'][n])):
                f.write(str(hdf['station'][n]) + 
                        ' is missing gust direction data at ' + 
                        str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking' + 
              ' for missing hourly data: ' + e.args[0])
    f.write('\n')
    f.write('\n')
        
    f.write('Temperature Data Report\n')
    f.write('-----------------------\n')
    
    try:                
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['tmean'][n] > 46.1:
                f.write(str(hdf['station'][n]) + 
                        ' reported a temperature of ' + str(hdf['tmean'][n]) + 
                        '°C on ' + str(hdf['date'][n]) + 
                        '. If true, this a new state record for max temp.\n')
            elif hdf['tmean'][n] < -51.7:
                f.write(str(hdf['station'][n]) + 
                        ' reported a temperature of ' + str(hdf['tmean'][n]) +
                        '°C on ' + str(hdf['date'][n]) + 
                        '. If true, this a new state record for min temp.\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly temperature data:' + e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Relative Humidity Data Report\n')
    f.write('-----------------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['rh'][n] > 1.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported a relative humidity of ' + str(hdf['rh'][n]) +
                        ' at ' + str(hdf['date'][n]) + '\n')
            elif hdf['rh'][n] < 0.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported negative relative humidity at ' +
                       str(hdf['date'][n]) + '\n')
            elif hdf['rh'][n] < 0.01:
                f.write(str(hdf['station'][n]) + 
                        ' broke the world record for low relative humidity at ' + 
                       str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly relative humidity data: ' + 
             e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Vapor Pressure Data Report\n')
    f.write('--------------------------\n')
    #
    #
    #
    #
    
    f.write('\n')
    f.write('\n')
    
    f.write('Solar Radiation Data Report\n')
    f.write('---------------------------\n')
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['sr'][n] < 0.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported negative solar radiation at ' + 
                        str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly solar radiation data: ' +
             e.args[0])
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Speed Data Report\n')
    f.write('----------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['ws'][n] < 0.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported negative wind speed at ' + 
                        str(hdf['date'][n]) + '\n')
            elif hdf['ws'][0] > 77.785:
                f.write(str(hdf['station'][n]) + 
                        ' reported a record breaking wind speed at ' + 
                        str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly wind speed data: ' + 
             e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Wind Direction Data Report\n')
    f.write('--------------------------\n')
    
    try:
        n = 0 
        for index,rows in hdf.iterrows():
            if hdf['wind_vec'][n] < 0.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported negative wind direction at ' + 
                        str(hdf['date'][n]) + '\n')
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly wind direction data:' + 
             e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Precipitation Data Report\n')
    f.write('-------------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['pp'][n] > 51.562:
                f.write(str(hdf['station'][n]) + 
                        ' reported recording breaking amount of precip at ' + 
                        str(hdf['date'][n]) + '\n')
            elif hdf['pp'][n] < 0.0:
                f.write(str(hdf['station'][n]) + ' reported negative precip at ' + 
                        str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly precip data: ' + e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('5cm Soil Temperature Data Report\n')
    f.write('--------------------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['st5'][n] > 50.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported impossibly high 5cm soil temperature at ' +
                       str(hdf['date'][n]) + '\n')
            elif hdf['st5'][n] < -15.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported impossibly low 5cm soil temperature at ' + 
                       str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrhile while looking for' + 
              ' errors in hourly 5cm soil temperature: ' + 
             e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('15cm Soil Temperature Data Report\n')
    f.write('---------------------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['st15'][n] > 50.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported impossibly high 15cm soil temeprature at ' + 
                       str(hdf['date'][n]) + '\n')
            elif hdf['st15'][n] < -15.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported impossibly low 15cm soil temperature at ' + 
                       str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly 15cm soil temperature: ' + 
             e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Speed Data Report\n')
    f.write('----------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['gust'][n] < 0.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported negative gust speed at ' + 
                        str(hdf['date'][n]) + '\n')
            elif hdf['gust'][n] > 77.785:
                f.write(str(hdf['station'][n]) + 
                        ' reported a record breaking gust speed at ' + 
                        str(hdf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly gust speed data: ' + 
             e.args[0])
    
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Direction Data Report\n')
    f.write('--------------------------\n')
    
    try:
        n = 0
        for index,rows in hdf.iterrows():
            if hdf['gustdir'][n] < 0.0:
                f.write(str(hdf['station'][n]) + 
                        ' reported negative gust direction at ' + 
                        str(hdf['date'][n]) + '\n' )
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in hourly gust direction data: ' + 
             e.args[0])
    
    f.write('Daily Data\n')
    f.write('----------\n')
    f.write('Missing data:\n')
    f.write('-------------\n')     
    
    # Creates an empty dataframe into which all of the daily data from each region 
    # will be entered
    dydf = pd.DataFrame()
    
    # Uses the newly created 'station_list' to loop through
    # all of the stations in a given region and calls the 
    # 'QAQC_datapull' function to collect all of yesterdays daily data 
    # into a DataFrame
    for x in station_list:
        try:
            dydf = dydf.append(QAQC_datapull('daily',x,yesterday,yesterday))
        except ValueError:
            f.write(x + ' is missing all daily data\n')
    
    
    dydf = dydf.reset_index()
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if(pd.isnull(dydf['tave'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing daily average temperature data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['tmax'][n])):
                f.write(str(dydf['station'][n]) + 
                       ' is missing daily maximum temperature data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['tmin'][n])):
                f.write(str(dydf['station'][n]) + 
                   ' is missing daily minimum temperature data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['rhmax'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing daily maximum relative humidity data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['rhmin'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing daily minimum relative humidity data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['vp'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing vapor pressure data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['sr'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing solar radiation data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['st5mx'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing maximum daily 5cm soil temperature data at ' + 
                       str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['st5mn'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing minimum daily 5cm soil temperature data at ' + 
                       str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['st15mx'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing maximum daily 15cm soil temperature data at ' + 
                       str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['st15mn'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing minimum daily 15cm soil temperature data at ' + 
                       str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['pp'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing precipitation data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['gust'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing gust speed data at ' + 
                        str(dydf['date'][n]) + '\n')
            elif(pd.isnull(dydf['gustdir'][n])):
                f.write(str(dydf['station'][n]) + 
                        ' is missing gust direction data at ' + 
                        str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' daily missing data: ' + e.args[0])
   
    f.write('\n')
    f.write('\n')
        
    f.write('Temperature Data Report\n')
    f.write('-----------------------\n')
    
    try:                
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['tave'][n] > 46.1:
                f.write(str(dydf['station'][n]) + 
                        ' reported a average daily temperature of ' +
                        str(dydf['tave'][n]) + '°C on ' + str(dydf['date'][n]) + 
                        '. If true, this is a new state record for max temp.\n')
            elif dydf['tave'][n] < -51.7:
                f.write(str(dydf['station'][n]) + 
                        ' reported a averge daily temperature of ' + 
                        str(dydf['tave'][n]) + '°C on ' + str(dydf['date'][n]) +
                        '. If true, this is a new state record for min temp.\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily average temperature data: ' + 
             e.args[0])
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['tmax'][n] > 46.1:
                f.write(str(dydf['station'][n]) + 
                        ' reported a maximum daily temperature of ' +
                        str(dydf['tmax'][n]) + '°C on ' + str(dydf['date'][n]) + 
                        '. If true, this is a new state record for max temp.\n')
            elif dydf['tmax'][n] < -51.7:
                f.write(str(dydf['station'][n]) + 
                        ' reported a maximum daily temperature of ' + 
                        str(dydf['tmax'][n]) + '°C on ' + str(dydf['date'][n]) +
                        '. If true, this is a new state record for min temp.\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily maximum temperature data:' + 
             e.args[0])
        
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['tmin'][n] > 46.1:
                f.write(str(dydf['station'][n]) + 
                        ' reported a minimum daily temperature of ' +
                        str(dydf['tmin'][n]) + '°C on ' + str(dydf['date'][n]) + 
                        '. If true, this is a new state record for max temp.\n')
            elif dydf['tmin'][n] < -51.7:
                f.write(str(dydf['station'][n]) + 
                        ' reported a minimum daily temperature of ' + 
                        str(dydf['tmin'][n]) + '°C on ' + str(dydf['date'][n]) +
                        '. If true, this is a new state record for min temp.\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily minimum temperature data:' + 
             e.args[0])     

        
    f.write('\n')
    f.write('\n')
    
    f.write('Relative Humidity Data Report\n')
    f.write('-----------------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['rhmax'][n] > 1.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported a daily maximum relative humidity of ' +
                        str(dydf['rhmax'][n]) + ' at ' + str(dydf['date'][n]) + '\n')
            elif dydf['rhmax'][n] < 0.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported negative daily maximum relative humidity at ' +
                       str(dydf['date'][n]) + '\n')
            elif dydf['rhmax'][n] < 0.01:
                f.write(str(dydf['station'][n]) + 
                        ' broke the world record for low relative humidity at ' + 
                       str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e: 
        print('Something went wrong while looking for' + 
              ' errors in daily maximum relative' +
              ' humidity data: ' + e.args[0])
        
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['rhmin'][n] > 1.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported a daily minimum relative humidity of ' +
                        str(dydf['rhmin'][n]) + ' at ' + 
                        str(dydf['date'][n]) + '\n')
            elif dydf['rhmin'][n] < 0.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported negative daily minimum relative humidity at ' + 
                        str(dydf['date'][n]) + '\n')
            elif dydf['rhmin'][n] < 0.01:
                f.write(str(dydf['station'][n]) + 
                        ' broke the world record for low relative humidity at ' + 
                       str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e: 
        print('Something went wrong while looking for' + 
              ' errors in daily minimum relative' +
              ' humidity data: ' + e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Vapor Pressure Data Report\n')
    f.write('--------------------------\n')
    #
    #
    #
    #
    
    f.write('\n')
    f.write('\n')
    
    f.write('Solar Radiation Data Report\n')
    f.write('---------------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['sr'][n] < 0.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported negative solar radiation at ' + 
                        str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily solar radiation data: ' + 
             e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Precipitation Data Report\n')
    f.write('-------------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['pp'][n] > 51.562:
                f.write(str(dydf['station'][n]) + 
                        ' reported recording breaking amount of precip at ' + 
                        str(dydf['date'][n]) + '\n')
            elif dydf['pp'][n] < 0.0:
                f.write(str(dydf['station'][n]) + ' reported negative precip at ' + 
                        str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily precip data ' + e.argsa[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('5cm Soil Temperature Data Report\n')
    f.write('--------------------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['st5mx'][n] > 50.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly high daily maximum 5cm soil' +
                        ' temperature at ' + str(dydf['date'][n]) + '\n')
            elif dydf['st5mx'][n] < -15.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly low daily maximum 5cm soil' +
                        ' temperature at ' + str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for errors in daily maximum 5cm' +
              ' soil temperature data: ' + e.args[0])
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['st5mn'][n] > 50.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly high daily minimum 5cm soil' +
                        ' temperature at ' + str(dydf['date'][n]) + '\n')
            elif dydf['st5mn'][n] < -15.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly low daily minimum 5cm soil' +
                        ' temperature at ' + str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for errors in daily minimum 5cm' +
              ' soil temperature data: ' + e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('15cm Soil Temperature Data Report\n')
    f.write('---------------------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['st15mx'][n] > 50.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly high daily maximum 15cm soil' +
                        ' temeprature at ' + str(dydf['date'][n]) + '\n')
            elif dydf['st15mx'][n] < -15.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly low daily maximum 15cm soil' +
                        ' temperature at ' + str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily maximum 15cm' +
              ' soil temperature data: ' + e.args[0])
        
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['st15mn'][n] > 50.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly high daily minimum 15cm soil' +
                        ' temeprature at ' + str(dydf['date'][n]) + '\n')
            elif dydf['st15mn'][n] < -15.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported impossibly low daily minimum 15cm soil' +
                        ' temperature at ' + str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily minimum 15cm' +
              ' soil temperature data: ' + e.args[0])
        
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Speed Data Report\n')
    f.write('----------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['gust'][n] < 0.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported negative gust speed at ' + 
                        str(dydf['date'][n]) + '\n')
            elif dydf['gust'][n] > 77.785:
                f.write(str(dydf['station'][n]) + 
                        ' reported a record breaking gust speed at ' + 
                        str(dydf['date'][n]) + '\n')
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while looking for' + 
              ' errors in daily gust speed data: ' + e.args[0])
    
    f.write('\n')
    f.write('\n')
    
    f.write('Gust Direction Data Report\n')
    f.write('--------------------------\n')
    
    try:
        n = 0
        for index,rows in dydf.iterrows():
            if dydf['gustdir'][n] < 0.0:
                f.write(str(dydf['station'][n]) + 
                        ' reported negative gust direction at ' + 
                        str(dydf['date'][n]) + '\n' )
            n = n + 1
        f.write('\n')
    except Exception as e:
        print('Something went wrong while checking for' + 
              ' errors in daily gust direction data: ' +
              e.args[0])
        
    f.close()

In [8]:
for region in region_list:
    QC_report_by_region(region)